### What is Azure OpenAI Service?

Azure OpenAI Service provides REST API access to OpenAI's powerful language models including the GPT-3, Codex and Embeddings model series. In addition, the new GPT-4 and ChatGPT (gpt-35-turbo) model series have now reached general availability. These models can be easily adapted to your specific task including but not limited to content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or our web-based interface in the Azure OpenAI Studio.

#### Azure OpenAI Access

Access is currently limited as we navigate high demand, upcoming product improvements, and [Microsoft’s commitment to responsible AI](https://www.microsoft.com/ai/responsible-ai?activetab=pivot1:primaryr6). For now, we're working with customers with an existing partnership with Microsoft, lower risk use cases, and those committed to incorporating mitigations.

More specific information is included in the application form. We appreciate your patience as we work to responsibly enable broader access to Azure OpenAI.

Apply [here](https://aka.ms/oaiapply) for access

#### Key concepts - Prompts & Completions

The completions endpoint is the core component of the API service. This API provides access to the model's text-in, text-out interface. Users simply need to provide an input prompt containing the English text command, and the model will generate a text completion.

Here's an example of a simple prompt and completion:

Prompt: """ count to 5 in a for loop """

Completion: for i in range(1, 6): print(i)

#### Key concepts - Tokens

Azure OpenAI processes text by breaking it down into tokens. Tokens can be words or just chunks of characters. For example, the word “hamburger” gets broken up into the tokens “ham”, “bur” and “ger”, while a short and common word like “pear” is a single token. Many tokens start with a whitespace, for example “ hello” and “ bye”.

The total number of tokens processed in a given request depends on the length of your input, output and request parameters. The quantity of tokens being processed will also affect your response latency and throughput for the models.

#### Key concepts - In-context learning
The models used by Azure OpenAI use natural language instructions and examples provided during the generation call to identify the task being asked and skill required. When you use this approach, the first part of the prompt includes natural language instructions and/or examples of the specific task desired. The model then completes the task by predicting the most probable next piece of text. This technique is known as "in-context" learning. These models aren't retrained during this step but instead give predictions based on the context you include in the prompt.

There are three main approaches for in-context learning: Few-shot, one-shot and zero-shot. These approaches vary based on the amount of task-specific data that is given to the model:

- Few-shot: In this case, a user includes several examples in the call prompt that demonstrate the expected answer format and content. The number of examples typically range from 0 to 100 depending on how many can fit in the maximum input length for a single prompt. Maximum input length can vary depending on the specific models you use. Few-shot learning enables a major reduction in the amount of task-specific data required for accurate predictions. This approach will typically perform less accurately than a fine-tuned model.

- One-shot: This case is the same as the few-shot approach except only one example is provided.

- Zero-shot: In this case, no examples are provided to the model and only the task request is provided.

#### Pre-requisites

An Azure subscription - [Create one for free](https://azure.microsoft.com/free/cognitive-services)

Access granted to Azure OpenAI in the desired Azure subscription

Currently, access to this service is granted only by application. You can apply for access to Azure OpenAI by completing the form [here](https://aka.ms/oai/access).

[Python 3.7.1 or later version](https://www.python.org/)

The following Python libraries: os, requests, json, openai

An Azure OpenAI Service resource with a model deployed. For more information about model deployment, see the [resource deployment guide](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource).

In [ ]:
#Install the OpenAI Python client library with:
%pip install openai

#### Retrieve key and endpoint
To successfully make a call against the Azure OpenAI service, you'll need the following:

- ENDPOINT -> This value can be found in the Keys & Endpoint section when examining your resource from the Azure portal. Alternatively, you can find the value in Azure OpenAI Studio > Playground > Code View. An example endpoint is: https://docs-test-001.openai.azure.com/.
- API-KEY -> This value can be found in the Keys & Endpoint section when examining your resource from the Azure portal. You can use either KEY1 or KEY2.
- DEPLOYMENT-NAME -> This value will correspond to the custom name you chose for your deployment when you deployed a model. This value can be found under Resource Management > Deployments in the Azure portal or alternatively under Management > Deployments in Azure OpenAI Studio.

Go to your resource in the Azure portal. The Endpoint and Keys can be found in the Resource Management section. Copy your endpoint and access key as you'll need both for authenticating your API calls. You can use either KEY1 or KEY2. Always having two keys allows you to securely rotate and regenerate keys without causing a service disruption.

#### Environment variables
Copy .env.sample to .env and fill in the values for your environment.

- OPENAI_API_KEY=
- OPENAI_API_BASE=
- DEPLOYMENT_NAME_DAVINCI=


In [3]:
# Import Python libraries
import os
import openai
from Utilities.envVars import *

# Set OpenAI API key and endpoint
openai.api_type = "azure"
openai.api_version = OpenAiVersion
openai_api_key = OpenAiKey
assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key
openAiEndPoint = f"https://{OpenAiService}.openai.azure.com"
assert openAiEndPoint, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in openAiEndPoint.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = openAiEndPoint
davincimodel = OpenAiDavinci

#### Basic Completion

Let's look at a simple example to summarize a text. Remember that you can always control the output of a prompt via instructions, some typical controls are around:

- length of the summary
- target audience of the summary

In [4]:
# Give your prompt here
prompt = """Summarize this for a second-grade student:

        Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of 
        the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. 
        Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient
        civilizations since before recorded history. It is named after the Roman god Jupiter. 
        When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, 
        and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

try:
    # Create a completion for the provided prompt and parameters
    # To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
    completion = openai.Completion.create(
                    prompt=prompt,
                    temperature=0,
                    max_tokens=500,
                    engine=davincimodel)

    # print the completion
    print(completion.choices[0].text.strip(" \n"))
    
    # Here indicating if the response is filtered
    if completion.choices[0].finish_reason == "content_filter":
        print("The generated content is filtered.")
        
except openai.error.APIError as e:
    # Handle API error here, e.g. retry or log
    print(f"OpenAI API returned an API Error: {e}")

except openai.error.AuthenticationError as e:
    # Handle Authentication error here, e.g. invalid API key
    print(f"OpenAI API returned an Authentication Error: {e}")

except openai.error.APIConnectionError as e:
    # Handle connection error here
    print(f"Failed to connect to OpenAI API: {e}")

except openai.error.InvalidRequestError as e:
    # Handle connection error here
    print(f"Invalid Request Error: {e}")

except openai.error.RateLimitError as e:
    # Handle rate limit error
    print(f"OpenAI API request exceeded rate limit: {e}")

except openai.error.ServiceUnavailableError as e:
    # Handle Service Unavailable error
    print(f"Service Unavailable: {e}")

except openai.error.Timeout as e:
    # Handle request timeout
    print(f"Request timed out: {e}")

Jupiter is the fifth planet from the Sun and it is very big and bright. It is named after the Roman god Jupiter and it can be seen in the night sky. It is one of the brightest things you can see without a telescope.


In [5]:
#Let's change the prompt to specify the length of the summary to be 3 sentences.
prompt = """Summarize this for a second-grade student in 3 sentences:

        Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of 
        the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. 
        Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient
        civilizations since before recorded history. It is named after the Roman god Jupiter. 
        When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, 
        and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.Completion.create(
                prompt=prompt,
                temperature=0,
                max_tokens=500,
                engine=davincimodel)

# print the completion
print(completion.choices[0].text.strip(" \n"))

Jupiter is the fifth planet from the Sun and the biggest in the Solar System. It is very bright and can be seen in the night sky. It is named after the Roman god Jupiter and has been known since ancient times.


In [6]:
#Let's now change the prompt to specify the target audience of the summary to be a physicist.
prompt = """Summarize this for a physicist in 3 sentences:

        Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of 
        the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. 
        Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient
        civilizations since before recorded history. It is named after the Roman god Jupiter. 
        When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, 
        and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.Completion.create(
                prompt=prompt,
                temperature=0,
                max_tokens=500,
                engine=davincimodel)

# print the completion
print(completion.choices[0].text.strip(" \n"))

Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass two-and-a-half times that of all the other planets combined and is visible to the naked eye in the night sky. It is named after the Roman god Jupiter and is the third-brightest natural object in the night sky after the Moon and Venus.


In [7]:
#You can easily control the format of the summarization as well to fit into desired formats. 
#Let's summarize the same text in a bullet point format.
prompt = """Summarize this for a second-grade student in 3 sentences in bullet points:

        Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of 
        the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. 
        Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient
        civilizations since before recorded history. It is named after the Roman god Jupiter. 
        When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, 
        and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.Completion.create(
                prompt=prompt,
                temperature=0,
                max_tokens=500,
                engine=davincimodel)

# print the completion
print(completion.choices[0].text.strip(" \n"))

- Jupiter is the fifth planet from the Sun and the largest in the Solar System. 
- It is very bright and can be seen in the night sky. It is named after the Roman god Jupiter. 
- When viewed from Earth, Jupiter is the third-brightest natural object in the night sky after the Moon and Venus.


In [8]:
#Summarize this for a second-grade student as a title and a one sentence abstract:
prompt = """Summarize this for a second-grade student as a title and a one sentence abstract:

        Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of 
        the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. 
        Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient
        civilizations since before recorded history. It is named after the Roman god Jupiter. 
        When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, 
        and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.Completion.create(
                prompt=prompt,
                temperature=0,
                max_tokens=500,
                engine=davincimodel)

# print the completion
print(completion.choices[0].text.strip(" \n"))

Title: Jupiter, the Brightest Planet in the Solar System
Abstract: Jupiter is the fifth planet from the Sun and the brightest object in the night sky, known to ancient civilizations since before recorded history.


In [9]:
#Sentiment Analysis
prompt = """Decide whether the following customer feedback is positive or negative.

        Q: I was disappointed with the quality of the product. It was very cheaply made and did not meet my expectations at all.
        Q: I was happy with this product, it is well made and great quality for the price.
        """

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.Completion.create(
                prompt=prompt,
                temperature=0,
                max_tokens=500,
                engine=davincimodel)

# print the completion
print(completion.choices[0].text.strip(" \n"))

Negative: I was disappointed with the quality of the product.

Positive: I was happy with this product, it is well made and great quality for the price.


In [10]:
#PII Detection
prompt =  """List all PII data from following statement:
        John Doe is a 35-year old man and he lives at 21 Main Street, New York, NY. He is a software engineer and he works at Google. He has a wife named Jane Doe and they have two children
        """

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.Completion.create(
                prompt=prompt,
                temperature=0,
                max_tokens=500,
                engine=davincimodel)

# print the completion
print(completion.choices[0].text.strip(" \n"))

PII Data:
- John Doe
- 35-year old
- 21 Main Street, New York, NY
- Jane Doe
- Two children


In [11]:
#Code Generation
prompt =  """### Postgres SQL tables, with their properties:
        #
        # Employee(id, name, department_id)
        # Department(id, name, address)
        # Salary_Payments(id, employee_id, amount, date)
        #
        ### A query to list the names of the departments which employed more than 10 employees in the last 3 months
        query: SELECT"""

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.Completion.create(
                prompt=prompt,
                temperature=0,
                max_tokens=500,
                engine=davincimodel)

# print the completion
print(completion.choices[0].text.strip(" \n"))

d.name 
        FROM Department d 
        INNER JOIN Employee e 
        ON d.id = e.department_id 
        INNER JOIN Salary_Payments sp 
        ON e.id = sp.employee_id 
        WHERE sp.date > NOW() - INTERVAL '3 months' 
        GROUP BY d.name 
        HAVING COUNT(*) > 10;


In [12]:
#Code Explanation
prompt =  """Can you explain what does this code do?
        #
        # ###
        Code:
        SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id 
        FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10
        #
        #
        Answer:
        #"""

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.Completion.create(
                prompt=prompt,
                temperature=0,
                max_tokens=500,
                engine=davincimodel)

# print the completion
print(completion.choices[0].text.strip(" \n"))

This code is a SQL query that will select the name of each department that has more than 10 employees who have received salary payments in the last 3 months. It does this by joining the Department and Employee tables on the department_id field, then selecting the employee_id from the Salary_Payments table where the date is within the last 3 months. Finally, it groups the results by department name and only returns departments that have more than 10 employees.


In [13]:
# Helper Function to call OpenAI
def callOpenAi(num_times, prompt, temperature):
    for i in range(num_times):
        response = openai.Completion.create(
            engine=davincimodel,
            prompt=prompt,
            max_tokens=60,
            temperature = temperature
        )
        print(response['choices'][0]['text'])

In [14]:
# Temperature - Defaults to 1
# What sampling temperature to use. Higher values means the model will take more risks. 
# Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.
#We generally recommend altering this or top_p but not both.
callOpenAi(10, 'The best pet is a ', temperature = 0)


dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, 

In [15]:
# Let's vary the temperature and set it to "1"
callOpenAi(10, 'The best pet is a ', temperature = 1)



dog. Dogs are loyal, intelligent, loving, and make great companions. They can also provide protection, exercise, stress and anxiety relief, and unconditional love.

dog. Dogs are loyal, loving companions and can also provide protection for their families. They are easy to train and have a long lifespan. They can also provide great companionship and help reduce stress and anxiety.

dog.
Dogs are incredibly loyal and loving animals that form bonds with their owners. They make great companions and can help with depression and anxiety. Dogs are also very active and can be trained to do a variety of activities, from playing catch to performing tricks. Plus, they're great for


dog.

dog.

dog. Dogs are highly intelligent, loyal, playful and generally loving companions. They can provide protection from intruders and help keep the family entertained. They are also generally trainable and willing to learn new tricks and tasks.


cat. Cats make great companions, are relatively independent, an

In [16]:
def callOpenAiTopP(num_times, prompt, top_p):
    for i in range(num_times):
        
        response = openai.Completion.create(
            engine=davincimodel,
            prompt=prompt,
            max_tokens=60,
            top_p = top_p
        )
        print(response['choices'][0]['text'])

In [17]:
# top_p - Defaults to 1
# An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens 
# with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
# We generally recommend altering this or temperature but not both.
callOpenAiTopP(10, 'The best pet is a ', top_p = 1)


dog. Dogs are loyal, friendly, and make great companions. They can provide companionship, help with security, and offer unconditional love and affection.

dog. Dogs are loyal companions, loving and affectionate, and can even help you stay fit and active. Plus, their unconditional love always brightens up your day.

dog. Dogs are very loving, loyal, and have a strong bond with humans that provides comfort and security. They also require regular exercise, which gives people an opportunity to get outdoors and stay healthy. Additionally, dogs can provide companionship and unconditional love, which can help reduce stress and improve mental

dog



dog. Dogs make great companions and offer companionship, unconditional love, loyalty and protection. They can also be trained to do many things, such as obedience commands, activity games and therapeutic tasks.


cat. Cats make great pets as they are low-maintenance, playful, affectionate, and fairly independent. Plus, cats are clean and can keep

In [18]:
# Let's vary the top_p and set it to "0.1"
callOpenAiTopP(10, 'The best pet is a ', top_p = 0.1)


dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, loving, and make great companions. They can be trained to do a variety of tasks and can provide a great source of comfort and companionship.

dog. Dogs are loyal, 